### Understanding the poisson solver

I want to understand how the Poisson equation $\nabla^2 \phi = \frac{S(\vec{x} )}{k}$ is solved in underworld3.

In [1]:
import underworld3 as uw

In [2]:
## create our mesh
mesh = uw.meshing.UnstructuredSimplexBox(minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize= 1.0 / (24), qdegree=5)

In [3]:
## define our field on the mesh
phi = uw.discretisation.MeshVariable(r"\phi", mesh, 1, degree=2)

I'll try use the Poisson solver to solve the Poisson equation

In [4]:
poisson_solver = uw.systems.Poisson(mesh, u_Field=phi)

staring the Poisson solver


As far as I can tell, the poisson solver uses the flux $\vec{F} = k \nabla \phi$. Once we have this flux, uw3 in the background can write an equation of the form $\nabla \cdot \vec{F} - f_0 = 0$. If we have $f_0 = S( \vec{x})$, then this is the equation $\nabla \cdot (k \nabla \phi) = S (
\vec{x})$. If $k$ is constant in space, then this is the original equation. uw3 can then go away and convert this into a weak form and then solve using finite element methods.

To define the flux $F$, we use a consitutive diffusion model and set the diffusivity in this model to $k$

In [5]:
poisson_solver.constitutive_model = uw.systems.constitutive_models.DiffusionModel(mesh.dim)
poisson_solver.constitutive_model.Parameters.diffusivity = 1 ## we will make it a constant for now

To add the source term $S(\vec{x})$, we change the poisson_solver.f property. For simplicity here, lets set $S(\vec{x}) = 0$. 

In [6]:
poisson_solver.f = 0 

Then, we can give the solver some boundary conditions, namely $\phi(x,0) = 1$ and $\phi(x, 1) = 0$.

In [7]:
poisson_solver.add_dirichlet_bc(1, "Bottom")
poisson_solver.add_dirichlet_bc(0, "Top")

We can now solve the poisson equation $\nabla^2 \phi = 0$

In [8]:
poisson_solver.solve()

running the poisson discription
Matrix([[0]])
Matrix([[0]])
Matrix([[0]])
Matrix([[0, 0]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 11.2336 
  1 SNES Function norm 0.000501603 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


Store our solution to the problem

In [9]:
with mesh.access():
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data)

Now, lets print everything out so that we can see our result

In [10]:
from mpi4py import MPI ## library for displaying everything

In [11]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phi"] = mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phi",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="test.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/3501045330.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='1f8...], height=500, min_width=500, sizing_mode='stretch_width')

And thats our solution to Laplaces equation $\nabla^2 \phi = 0$ on $[0,1] \times [0,1]$ with $\phi(x,0) = 1, \phi(x,1)= 0$

Now, lets understand what happens if we change the variable $k$. If we set $k=0$, then the flux term term $\vec{F} = k \nabla \phi = 0$. Then, the whole solution does not evolve as there is no flux term. Thus, we get no change from the initial conditions.

Lets plot that

Set $k=0$:

In [12]:
poisson_solver.constitutive_model.Parameters.diffusivity = 0

Keep our bc and source function

In [13]:
poisson_solver.add_dirichlet_bc(1, "Bottom")
poisson_solver.add_dirichlet_bc(0, "Top")
poisson_solver.f = 0

Solve our poisson_solver

In [14]:
poisson_solver.solve()

running the poisson discription
Matrix([[0]])
Matrix([[0]])
Matrix([[0]])
Matrix([[0, 0]])
Matrix([[0, 0]])
Matrix([[0, 0]])
at the end
  0 SNES Function norm < 1.e-11
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_ABS iterations 0


In [15]:
with mesh.access():
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data)
    
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phi"] = mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phi",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="test.png")
    

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/1320458023.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='fd7...], height=500, min_width=500, sizing_mode='stretch_width')

We can see that there is no propigation of the boundary, even though we are solving the same equation.

Now lets solve a more complicated problem with a source term, $\nabla^2 \phi = \sin(2 \pi x) \cos(2 \pi y)$. We will do this on the same domain with boundary conditions $\phi(0,y)=\phi(1,y)=0$ and $\phi(x,0) = \phi(x,1)= - \frac{\sin(2 \pi  x)}{8 \pi^2}$. To do this, we set $k=1$ and $S(\vec{x}) = \sin(2 \pi x) \cos(2 \pi y)$

In [16]:
poisson_solver.constitutive_model.Parameters.diffusivity = 1

To set the source term $S(\vec{x})$ we must make poisson_solver.f $\sin(2 \pi x) \cos(2 \pi y)$. To make it a function of the cooridnates we get the mesh coodinates using mesh.X. We pass the function poisson_solve.f in symbolically using a sympy expression.

In [17]:
import math
import sympy

x,y = mesh.X ## access the expressions for the coordinates of the mesh
source_term = sympy.sin( 2* sympy.pi * x) * sympy.cos(2* sympy.pi * y) ## write the expression for sin(x) cos(y) using sympy

You can see the source term is a symbolic expression

In [18]:
source_term

sin(2*N.x*pi)*cos(2*N.y*pi)

In [19]:
poisson_solver.f = -source_term

In [20]:
poisson_solver._f0

Matrix([[0]])

In [21]:
poisson_solver._f1

Matrix([[0, 0]])

We give the boundary conditions on the edges of the domain

In [22]:
top_boundary = - sympy.sin(2 * sympy.pi * x)/(8 * sympy.pi**2)
bottom_boundary = - sympy.sin(2 * sympy.pi * x)/(8 * sympy.pi**2)

poisson_solver.add_dirichlet_bc(0, "Left")
poisson_solver.add_dirichlet_bc(0, "Right") 
poisson_solver.add_dirichlet_bc(top_boundary, "Top")
poisson_solver.add_dirichlet_bc(bottom_boundary, "Bottom")

Now we solve the system

In [23]:
poisson_solver.solve()

running the poisson discription
Matrix([[sin(2*N.x*pi)*cos(2*N.y*pi)]])
Matrix([[0]])
Matrix([[-sin(2*N.x*pi)*cos(2*N.y*pi)]])
Matrix([[0, 0]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 0.143641 
  1 SNES Function norm 4.56464e-06 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


In [24]:
with mesh.access():
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data)
    
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phi"] = mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phi",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="numerical.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/310347640.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='e20...], height=500, min_width=500, sizing_mode='stretch_width')

The Poisson equation $\nabla^2 \phi = \sin(2 \pi x) \cos( 2 \pi y)$ with these boundary conditions has the analytic solution $\phi = -\frac{\cos(2 \pi y) \sin(2 \pi x)}{8 \pi^2}$. Lets now plot that and see how it compares to our solution

In [25]:
analytic_fn = -sympy.cos(2 * sympy.pi * y) * sympy.sin(2 * sympy.pi * x)/(8 * sympy.pi**2)
analytic_fn

-sin(2*N.x*pi)*cos(2*N.y*pi)/(8*pi**2)

In [26]:

with mesh.access():
    mesh_analytic_soln = uw.function.evaluate(analytic_fn, mesh.data)

In [27]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phiAnalytic"] = mesh_analytic_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phiAnalytic",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="analytic.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/3812309197.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='96b...], height=500, min_width=500, sizing_mode='stretch_width')

Lets looks at the difference here, why is this happening?


In [28]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["diff"] = mesh_analytic_soln - mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="diff",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="difference.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/847708945.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='dc8...], height=500, min_width=500, sizing_mode='stretch_width')

The analytic and numerical results match closely

So far, we have dealt with an equation of the form $\nabla \cdot (k \nabla \phi) = S(\vec{x})$, where $k$ is a constant in space. Now, we promote $k$ to a function of space $k(\vec{x})$. Lets solve $\nabla \cdot ((x+y) \nabla \phi) = 2 \pi  ((1-2 \pi  (x+y)) \cos (2
   \pi  x)-(2 \pi  (x+y)+1) \sin
   (2 \pi  x))$ on $[0,1]\times[0,1]$ with the boundary conditions 
   
   $\phi(x,1) = \phi(x,0) = \cos(2 \pi x) + \sin(2 \pi x), \phi(0,y) = \phi(1,y)=1$
   
To do this, we set $k(x,y) = x+y$ and $S(\vec{x}) = 2 \pi  ((1-2 \pi  (x+y)) \cos (2
   \pi  x)-(2 \pi  (x+y)+1) \sin
   (2 \pi  x))$
  

To write the diffusivity as a function of space, we access the symbolic mesh coordinates x and y

In [29]:
x, y = mesh.X

We can then write the diffusivity as a symbolic sympy function k_fn

In [30]:
k_fn = x + y

In [31]:
poisson_solver.constitutive_model.Parameters.diffusivity = k_fn

We can similariy write the source function

In [32]:
source_fn = 2 * sympy.pi *( (1 - 2 * sympy.pi *(x + y)) * sympy.cos(2 * sympy.pi * x) - (2 * sympy.pi * (x + y) + 1)*sympy.sin(2 * sympy.pi *x) ) 
source_fn

2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))

In [33]:
poisson_solver.f = -source_fn

Add in the boundary conditions

In [34]:
top_condition    = sympy.cos(2 * sympy.pi * x) + sympy.sin(2 * sympy.pi * x) 
bottom_condition = sympy.cos(2 * sympy.pi * x) + sympy.sin(2 * sympy.pi * x) 
left_condition   = 1
right_condition  = 1

poisson_solver.add_dirichlet_bc( top_condition, "Top")
poisson_solver.add_dirichlet_bc( bottom_condition, "Bottom")
poisson_solver.add_dirichlet_bc( left_condition, "Left")
poisson_solver.add_dirichlet_bc( right_condition, "Right")


Solve

In [35]:
poisson_solver.solve()

running the poisson discription
Matrix([[2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0]])
Matrix([[-2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0, 0]])
Matrix([[(N.x + N.y)*\phi_{,0}(N.x, N.y), (N.x + N.y)*\phi_{,1}(N.x, N.y)]])
Matrix([[(N.x + N.y)*\phi_{,0}(N.x, N.y), (N.x + N.y)*\phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 25.575 
  1 SNES Function norm 0.000943213 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


Now, lets plot our numerical solution

In [36]:
with mesh.access():
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data)
    
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phi"] = mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phi",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="numericalWithNonConstantK.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/3619014333.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='727...], height=500, min_width=500, sizing_mode='stretch_width')

The analytical solutuon for this is $\phi = \sin(2 \pi x) + \cos(2 \pi x)$. Lets plot it.

In [37]:
analytic_fn = sympy.sin(2 * sympy.pi * x) + sympy.cos(2 * sympy.pi *x)
analytic_fn

sin(2*N.x*pi) + cos(2*N.x*pi)

In [38]:
with mesh.access():
    mesh_analytic_soln = uw.function.evaluate(analytic_fn, mesh.data)
    
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phiAnalytic"] = mesh_analytic_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phiAnalytic",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="analyticWithNonConstantK.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/1678708188.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='d03...], height=500, min_width=500, sizing_mode='stretch_width')

And again we can plot the difference between the analytic and numerical results

In [39]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["diff"] = mesh_analytic_soln  - mesh_numerical_soln
    

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="diff",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="diffWithNonConstantK.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/3497995276.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='10d...], height=500, min_width=500, sizing_mode='stretch_width')

Lets now consider when the diffusivity $k$ is a function of the field $\phi$. Lets solve $\nabla \cdot ( \phi \nabla \phi) = -4 \pi ^2 y^2 \sin ^2(2 \pi  x)+4
   \pi ^2 y^2 \cos ^2(2 \pi 
   x)+\sin ^2(2 \pi  x)$ using the poisson solver on $[0,1]\times[0,1]$ with boundary conditions $\phi(x,0) = \phi(0,y) = \phi(1,y) = 0, \phi(x,1) = \sin(2 \pi x)$

To model this, we set $k = \phi$. We do this by accessing the symbolic repressentation of $\phi$ using sympy. 

In [40]:
k_fn = phi.sym[0]
k_fn

\phi(N.x, N.y)

Setting the diffusivity

In [41]:
poisson_solver.constitutive_model.Parameters.diffusivity = k_fn

Setting the source function

In [42]:
source_fn = -4 * sympy.pi**2 * y**2 * (sympy.sin(2 * sympy.pi *  x))**2+ 4 * sympy.pi**2 * y**2 * (sympy.cos(2 * sympy.pi * x))**2 +  (sympy.sin(2 * sympy.pi * x))**2
source_fn

-4*N.y**2*pi**2*sin(2*N.x*pi)**2 + 4*N.y**2*pi**2*cos(2*N.x*pi)**2 + sin(2*N.x*pi)**2

Setting the boundary conditions

In [43]:
top_condition    = sympy.sin(2 * sympy.pi * x)
bottom_condition = 0
left_condition   = 0
right_condition  = 0

poisson_solver.add_dirichlet_bc( top_condition, "Top")
poisson_solver.add_dirichlet_bc( bottom_condition, "Bottom")
poisson_solver.add_dirichlet_bc( left_condition, "Left")
poisson_solver.add_dirichlet_bc( right_condition, "Right")

Solve the equations

In [44]:
##a = 1
##poisson_solver.constitutive_model.Parameters.diffusivity = a + (1 - a) * k_fn
poisson_solver.solve()

running the poisson discription
Matrix([[2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0]])
Matrix([[-2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0, 0]])
Matrix([[\phi(N.x, N.y)*\phi_{,0}(N.x, N.y), \phi(N.x, N.y)*\phi_{,1}(N.x, N.y)]])
Matrix([[\phi(N.x, N.y)*\phi_{,0}(N.x, N.y), \phi(N.x, N.y)*\phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 3.78136 
Nonlinear Poisson_1_ solve did not converge due to DIVERGED_LINEAR_SOLVE iterations 0


In [45]:

analytic_solution = y*sympy.sin(2 * sympy.pi * x)
with mesh.access(phi):
    phi.data[:,0] = uw.function.evaluate(analytic_solution, coords=phi.coords, coord_sys=mesh.N)

In [46]:
poisson_solver.constitutive_model.Parameters.diffusivity = k_fn
poisson_solver.solve(zero_init_guess=False) 

running the poisson discription
Matrix([[2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0]])
Matrix([[-2*pi*((-2*pi*(N.x + N.y) + 1)*cos(2*N.x*pi) - (2*pi*(N.x + N.y) + 1)*sin(2*N.x*pi))]])
Matrix([[0, 0]])
Matrix([[\phi(N.x, N.y)*\phi_{,0}(N.x, N.y), \phi(N.x, N.y)*\phi_{,1}(N.x, N.y)]])
Matrix([[\phi(N.x, N.y)*\phi_{,0}(N.x, N.y), \phi(N.x, N.y)*\phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 0.9277 
Nonlinear Poisson_1_ solve did not converge due to DIVERGED_LINEAR_SOLVE iterations 0


In [47]:
for i in range(100):
    
    a -= 0.01
    print(a)
    poisson_solver.constitutive_model.Parameters.diffusivity = a + (1 - a)* k_fn
    poisson_solver.solve(zero_init_guess=False) 

NameError: name 'a' is not defined

Well that didnt work. Perhaps the non-linearity is too much for the solver?? It might be that we should pick another, easier example. Lets do that.

Lets now consider when the diffusivity $k$ is a function of the field $\phi$. Lets solve $\nabla \cdot ( (1 + \phi^2) \nabla \phi) = 12 (x^2 + y^2)^2$ using the poisson solver on $[0,1]\times[0,1]$ with boundary conditions $\phi(x,0) = x^2, \phi(x,1) = 1 + x^2, \phi(0,y) = y^2, \phi(1,y) = 1 + y^2$. To do this, we set $k=1+\phi^2$ and set our source term $S(\vec{x}) = 12 (x^2 + y^2)^2$.

We first set $k = 1 + \phi^2$. We write $k$ symoblically using sympy. 

In [48]:
k_fn = 1 + phi.sym[0]**2
k_fn

\phi(N.x, N.y)**2 + 1

In [49]:
poisson_solver.constitutive_model = uw.systems.constitutive_models.DiffusionModel(mesh.dim)
poisson_solver.constitutive_model.Parameters.diffusivity = k_fn

Set the boundary conditions

In [50]:
top_condition    = 1 + x**2
bottom_condition = x**2
left_condition   = y**2
right_condition  = 1+y**2

poisson_solver.add_dirichlet_bc( top_condition, "Top")
poisson_solver.add_dirichlet_bc( bottom_condition, "Bottom")
poisson_solver.add_dirichlet_bc( left_condition, "Left")
poisson_solver.add_dirichlet_bc( right_condition, "Right")

Set the source function $S(\vec{x}) = 4 (1 + 3 x^4 + 6 x^2 y^2 + 3 y^4)$ symbolically

In [51]:
x,y = mesh.X
source_fn = 4 * (1 + 3 * x**4 + 6 * x**2 * y **2 + 3 * y**4)
poisson_solver.f = -source_fn

For some non-linear problems, the poisson solver requires an initial guess for the field suffeciently close to the solution. To make this initial guess, we solve the system with weak non-linearity ($k = \alpha + (1 - \alpha) \phi$, $\alpha << 1$), get the result, and use this as the initial guess as we vary $a$ to $1$ resulting in $k = \phi$. This is shown below.

In [52]:
aList = [1- da/10 for da in range(11)] ## list of as
for a in aList: ## for each a parameter
    print(a) ## what a are we using
    ## set the diffusivity to according to a
    poisson_solver.constitutive_model.Parameters.diffusivity = a + (1 - a)* k_fn
    ## run the poisson solver using an initial guess output from the solution previous
    poisson_solver.solve(zero_init_guess=False)

1.0
running the poisson discription
Matrix([[12*N.x**4 + 24*N.x**2*N.y**2 + 12*N.y**4 + 4]])
Matrix([[0]])
Matrix([[-12*N.x**4 - 24*N.x**2*N.y**2 - 12*N.y**4 - 4]])
Matrix([[0, 0]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
Matrix([[\phi_{,0}(N.x, N.y), \phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 25.0407 
  1 SNES Function norm 0.00149411 
Nonlinear Poisson_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
0.9
running the poisson discription
Matrix([[12*N.x**4 + 24*N.x**2*N.y**2 + 12*N.y**4 + 4]])
Matrix([[0]])
Matrix([[-12*N.x**4 - 24*N.x**2*N.y**2 - 12*N.y**4 - 4]])
Matrix([[0, 0]])
Matrix([[(0.1*\phi(N.x, N.y)**2 + 1.0)*\phi_{,0}(N.x, N.y), (0.1*\phi(N.x, N.y)**2 + 1.0)*\phi_{,1}(N.x, N.y)]])
Matrix([[(0.1*\phi(N.x, N.y)**2 + 1.0)*\phi_{,0}(N.x, N.y), (0.1*\phi(N.x, N.y)**2 + 1.0)*\phi_{,1}(N.x, N.y)]])
at the end
  0 SNES Function norm 0.0924503 
  1 SNES Function norm 0.000789718 
  2 SNES Function norm 8.05033e-08 
Nonlinear Poisson_1_ solve 

The analytic solution to our poisson equation with the given boundary conditions is $\phi = x^2 + y^2$. Lets plot the analytic solution, the numerical solution and the difference between these.

In [53]:
x,y = mesh.X
analytic_fn = ( x**2 + y**2)

with mesh.access():
    mesh_analytic_soln = uw.function.evaluate(analytic_fn, mesh.data)
    mesh_numerical_soln = uw.function.evaluate(poisson_solver.u.fn, mesh.data)

In [54]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phiNumerical"] = mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phiNumerical",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="analyticWithNonConstantK.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/2277713793.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='4b7...], height=500, min_width=500, sizing_mode='stretch_width')

In [55]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["phiAnalytic"] = mesh_analytic_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="phiAnalytic",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="analyticWithNonConstantK.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/1661971018.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='819...], height=500, min_width=500, sizing_mode='stretch_width')

The analytic and numerical solutions are identical

In [56]:
if MPI.COMM_WORLD.size == 1:

    import numpy as np
    import pyvista as pv

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [500, 500]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("ignore_mesh.vtk")
    pvmesh = pv.read("ignore_mesh.vtk")

    pvmesh.point_data["diff"] = mesh_analytic_soln  - mesh_numerical_soln

    sargs = dict(interactive=True)  # doesn't appear to work :(
    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="diff",
        use_transparency=False,
        opacity=0.5,
        scalar_bar_args=sargs,
    )

    pl.camera_position = "xy"

    pl.show(cpos="xy")
    pl.screenshot(filename="diff.png")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_92690/2995387564.py:9: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, color_mappers=[LinearColorMapper(id='5cc...], height=500, min_width=500, sizing_mode='stretch_width')

# Poisson in non-cartesian Geometries

We have solved the poisson equation in cartesian coordiantes, but we can solve it in different geometries aswell. Lets solve $\nabla^2 \phi = r$ in a 2-dimensional annulus.

In [ ]:
import underworld3 as uw

In [ ]:
mesh = uw.meshing.SphericalShell(radiusOuter=1, radiusInner=0.1, cellSize=1/(24), qdegree=3)